In [ ]:
# importing all the required libraries for the project

import os
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, GaussianNoise, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

print("Number of GPUs available:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
data_dir = 'Q:/AI Projects/Datasets/Brain Tumor MRI Dataset/training'
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

for i in categories: # testing out how the paths for the images work in the dataset
    path = os.path.join(data_dir, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img))

In [ ]:
plt.imshow(img_array); # just printing the last image img_array was set to

In [ ]:
# the shape of an image in the dataset
img_array.shape

In [ ]:
# printing out 4 images of each class to take a look

plt.figure(figsize=(20, 16))

images_path = ['/glioma/Tr-glTr_0000.jpg', '/meningioma/Tr-meTr_0000.jpg', '/notumor/Tr-noTr_0000.jpg', '/pituitary/Tr-piTr_0000.jpg']

for i in range(4):
    ax = plt.subplot(2, 2, i + 1)
    img = cv2.imread(data_dir + images_path[i])
    img = cv2.resize(img, (250, 250))
    plt.imshow(img)
    plt.title(categories[i])

In [ ]:
# building a quick and simple model

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 1), activation='relu')) # our input layer
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu')) # convolutional layers that perform feature extraction
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(units=4096, activation='relu')) # dense layers that perform classification
model.add(Dropout(0.2))
model.add(GaussianNoise(0.2))

model.add(Dense(units=4096, activation='relu'))
model.add(Dropout(0.2))
model.add(GaussianNoise(0.2))

model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=4, activation='softmax')) # our output layer

model.summary()

In [ ]:
# compiling our model with the Adam optimizer

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, clipvalue=0.5)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics= ['categorical_accuracy'])

In [ ]:
# using ImageDataGenerator to prepare the images (resize, normalize, etc)

generator_train = ImageDataGenerator(rescale=1./255,
                                     featurewise_center=False,
                                     samplewise_center=False,
                                     featurewise_std_normalization=False,
                                     samplewise_std_normalization=False,
                                     zca_whitening=False,
                                     rotation_range=0,
                                     zoom_range = 0,
                                     width_shift_range=0,
                                     height_shift_range=0,
                                     horizontal_flip=True,
                                     vertical_flip=False) 

generator_test = ImageDataGenerator(rescale=1./255,
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=0,
                                    zoom_range = 0,
                                    width_shift_range=0,
                                    height_shift_range=0,
                                    horizontal_flip=True,
                                    vertical_flip=False)

# creating the train and test data

train = generator_train.flow_from_directory('Q:/AI Projects/Datasets/Brain Tumor MRI Dataset/training', target_size=(256, 256), batch_size=32, class_mode="categorical", color_mode='grayscale')
test = generator_test.flow_from_directory('Q:/AI Projects/Datasets/Brain Tumor MRI Dataset/testing', target_size=(256, 256), batch_size=32, class_mode="categorical", color_mode='grayscale')

In [ ]:
# creating callbacks for the model - if the model dosen't continue to improve (downward trend in loss not present) then training will stop

# stop training if loss doesn't keep decreasing
model_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)
model_rlr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, patience = 6, verbose = 1)

# automatically saves the best weights of the model - based on best val_accuracy
model_mcp = ModelCheckpoint(filepath = 'model_weights.h5', monitor = 'val_categorical_accuracy', save_best_only = True, verbose = 1)

# fitting the model to our data flowing from the data generator
history1 = model.fit(train, steps_per_epoch=5712//32, epochs=2, validation_data=test, validation_steps= 1311//32, callbacks=[model_es, model_rlr, model_mcp])

In [ ]:
print(history1.history.keys())

In [ ]:
# plotting model accuracy vs. epochs

plt.plot(history1.history['categorical_accuracy'])
plt.plot(history1.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# plotting model loss vs. epochs

plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()